In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mrmte\\Documents\\2CS_IASD\\SEDS\\mini-project\\mlops-mlflow-project\\mlops-end-to-end-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\mrmte\\Documents\\2CS_IASD\\SEDS\\mini-project\\mlops-mlflow-project\\mlops-end-to-end-project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mrmterki/mlops-end-to-end-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mrmterki"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d2a7c4bfb0226aca36651e19054196e1ee1d00c4"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/mrmterki/mlops-end-to-end-project.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet
import joblib
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.preprocessing import LabelEncoder

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Separate categorical and numerical columns
        categorical_cols = test_x.select_dtypes(include=['object']).columns
        numerical_cols = test_x.select_dtypes(exclude=['object']).columns

        # Impute categorical columns with the most frequent value
        categorical_imputer = SimpleImputer(strategy='most_frequent')
        test_x[categorical_cols] = categorical_imputer.fit_transform(test_x[categorical_cols])

        # Impute numerical columns with the mean value
        numerical_imputer = SimpleImputer(strategy='mean')
        test_x[numerical_cols] = numerical_imputer.fit_transform(test_x[numerical_cols])

        # Encode categorical 'ocean_proximity' column to numeric if it's still categorical
        label_encoder = LabelEncoder()
        test_x['ocean_proximity'] = label_encoder.fit_transform(test_x['ocean_proximity'])

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Register model to MLflow registry
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [11]:
import mlflow

mlflow.set_tracking_uri("https://dagshub.com/mrmterki/mlops-end-to-end-project.mlflow")
mlflow.set_registry_uri("https://dagshub.com/mrmterki/mlops-end-to-end-project.mlflow")

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Registry URI: {mlflow.get_registry_uri()}")


Tracking URI: https://dagshub.com/mrmterki/mlops-end-to-end-project.mlflow
Registry URI: https://dagshub.com/mrmterki/mlops-end-to-end-project.mlflow


In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-27 07:50:33,109: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-12-27 07:50:33,113: INFO: common: YAML file: params.yaml loaded successfully]
[2024-12-27 07:50:33,118: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-12-27 07:50:33,118: INFO: common: created directory at: artifacts]
[2024-12-27 07:50:33,118: INFO: common: created directory at: artifacts/model_evaluation]


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '<1H OCEAN'